# تحميل المكتبات الأساسية 

In [42]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os

# تفعيل استخدام ذاكرة بطاقة الرسومات GPU

In [26]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "وحدات معالجة الرسومات الفعلية,", len(logical_gpus), "وحدات معالجة الرسومات المنطقية")
    except RuntimeError as e:
        print(e)

# التحقق من توفر GPU

In [27]:
print("عدد وحدات معالجة الرسومات المتاحة: ", len(tf.config.list_physical_devices('GPU')))
print("الأجهزة الفعلية: ", tf.config.list_physical_devices())

عدد وحدات معالجة الرسومات المتاحة:  0
الأجهزة الفعلية:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


# تحديد المسار الأساسي للبيانات

In [29]:
base_dir = r'/home/abdennacer/Documents/GitHub/ML/kaggle/images'

# تعيين المسارات

In [30]:
#داخل هذه المجلدات أضف الصور بالقدر الذي يتوفر معك مع وضع كل مجموعة صور في مجلد يناسبها أو إنشاء مجلد اخر لها
#مع الانتباه لضرورة تطابق اسماء المجلدات الفرعية بين هذه المجلدات الثلاثة
#حاليا هناك ثلاث مجلدات فرعية فقط للإختبار، وهي كما يلي:
#تمور
#طاجين
#كسكس

train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

# إعداد المعاملات

In [31]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# تعزيز البيانات ومولدات البيانات

In [32]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 31 images belonging to 3 classes.
Found 12 images belonging to 3 classes.


# تحديد عدد الفئات

In [33]:
NUM_CLASSES = train_generator.num_classes

# بناء النموذج

In [34]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)  # استخدم 'softmax' لتصنيف متعدد الفئات

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


# تدريب النموذج

In [35]:
#إرفع هذه القيمة كلما توفرت لديك موارد تكفي لذلك، مع الأخذ بعين الاعتبار حجم البيانات او عدد الصور
epochs = 21
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // BATCH_SIZE,
    epochs=epochs
)

Epoch 1/21
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.1613 - loss: 1.2804 - val_accuracy: 0.1667 - val_loss: 1.2807
Epoch 2/21
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4839 - loss: 1.0080 - val_accuracy: 0.2500 - val_loss: 1.1628
Epoch 3/21
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.7097 - loss: 0.8204 - val_accuracy: 0.5833 - val_loss: 1.0704
Epoch 4/21
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.7419 - loss: 0.7192 - val_accuracy: 0.6667 - val_loss: 0.9837
Epoch 5/21
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.7742 - loss: 0.6275 - val_accuracy: 0.6667 - val_loss: 0.8949
Epoch 6/21
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.8387 - loss: 0.5572 - val_accuracy: 0.7500 - val_loss: 0.8083
Epoch 7/21
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 987ms/step - accuracy: 0.9032 - loss: 0.4312 - val_accuracy: 0.6667 - val_loss: 0.7320
Epoch 8/21
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 998ms/step - accuracy: 0.9355 - loss: 0.3372 - val_accuracy: 0.7500 - val_loss: 0.6678
Epoch 9/21

# تقييم النموذج

In [36]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
print(f'مستوى الخسارة: {loss}')
print(f'دقة الاختبار: {accuracy}')

Found 12 images belonging to 3 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step - accuracy: 0.6667 - loss: 0.6778
مستوى الخسارة: 0.6778225898742676
دقة الاختبار: 0.6666666865348816


# حفظ النموذج

In [37]:
model_save_path = os.path.join(base_dir, 'models/mobilenet_multilabel'+str(epochs)+'epoch.h5')
model.save(model_save_path)

# تحديد حجم الصورة

In [38]:
IMG_SIZE = (224, 224)

# مسار الصورة التي تريد اختبارها

In [39]:
image_path = r'/home/abdennacer/Documents/GitHub/ML/kaggle/couscousTest.jpg'

# تحميل الصورة ومعالجتها

In [43]:
def preprocess_image(image_path, target_size):
    img = load_img(image_path, target_size=target_size)  # تحميل الصورة
    img_array = img_to_array(img)  # تحويل الصورة إلى مصفوفة
    img_array = np.expand_dims(img_array, axis=0)  # إضافة بُعد الدفعة
    img_array /= 255.0  # تطبيع الصورة
    return img_array

# معالجة الصورة
image_array = preprocess_image(image_path, IMG_SIZE)

# إجراء التنبؤ

In [46]:
predictions = model.predict(image_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


# الحصول على تسميات الفئات

In [47]:
train_dir = r'/home/abdennacer/Documents/GitHub/ML/kaggle/images/train'
class_labels = sorted(os.listdir(train_dir))

# فك تشفير التنبؤات

In [52]:
predicted_class_indices = np.argmax(predictions, axis=1)
predicted_class_labels = [class_labels[i] for i in predicted_class_indices]

# طباعة احتمالات التنبؤ لجميع الفئات، مرتبة حسب الاحتمالية

In [53]:
prediction_probabilities = {class_labels[i]: prob for i, prob in enumerate(predictions[0])}
sorted_prediction_probabilities = sorted(prediction_probabilities.items(), key=lambda item: item[1], reverse=True)

print("احتمالات التنبؤ:")
for class_label, probability in sorted_prediction_probabilities:
    print(f"{class_label}: {probability:.4f}")

احتمالات التنبؤ:
tajin: 0.4984
koskos: 0.4613
tomor: 0.0403
